# Introduction: PUMS home value analysis

I am analyzing the 2011-2015 PUMS housing data, found [here](https://www.census.gov/programs-surveys/acs/data/pums.html). The data dictionary which explains the meaning of each column and the values contained within is available [here.](https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.2015.html) I want to find an algorithm that can predict the value of the home.

## Part I: Importing the data

We will start by importing some libraries very useful for machine learning including pandas and scikit-learn. It will be helpful to change the column names to be more readable. I have chosen to use data from two states with very low populations (North Dakota and Wyoming) and two states with very high populations (New York and Texas) to get a fairly diverse dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [2]:
dat = pd.concat([pd.read_csv('ss15hnd.csv'), pd.read_csv('ss15hny.csv'),
                 pd.read_csv('ss15hwy.csv'),
                 pd.read_csv('ss15htx.csv')], ignore_index = True)
dat.rename(columns = {"insp": "INSP"}, inplace = True)

In [3]:
colnames = ["ST", "NP", "ACR", "BATH", "BDSP", "ELEP", "GASP", "INSP",
              "RMSP", "RWAT", "SINK", "VALP", "VEH", "WATP", "YBL",
              "FINCP", "HINCP"]
readable_names = ["State", "Num_People", "Lot_Size", "Has_Bathtub", "Num_Bedrooms",
                  "Monthly_Electric", "Monthly_Gas", "Yearly_Insurance_Cost", "Num_Rooms",
                  "Has_Hot_Water", "Has_Sink", "Price", "Num_Vehicles",
                  "Yearly_Water", "Year_Built", "Family_Income", "Household_Income"]

cols = dat[colnames]

cols.columns = readable_names

cols.dropna(inplace = True)
cols.describe()

/Users/hunter/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,State,Num_People,Lot_Size,Has_Bathtub,Num_Bedrooms,Monthly_Electric,Monthly_Gas,Yearly_Insurance_Cost,Num_Rooms,Has_Hot_Water,Has_Sink,Price,Num_Vehicles,Yearly_Water,Year_Built,Family_Income,Household_Income
count,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,407700.000000,4.077000e+05,407700.000000,407700.000000,407700.000000,4.077000e+05,4.077000e+05
mean,43.520216,3.101925,1.299669,1.001683,3.316505,186.376954,59.355973,1134.024498,6.977479,1.002100,1.001688,2.537063e+05,2.255585,558.435847,5.277415,1.047924e+05,1.058923e+05
std,5.980571,1.356574,0.569388,0.040985,0.934867,106.674852,83.926798,1061.516282,2.264988,0.045773,0.041045,3.562430e+05,0.961443,578.861019,3.101509,1.021289e+05,1.023201e+05
min,36.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000e+02,0.000000,1.000000,1.000000,-1.680000e+04,-1.680000e+04
25%,36.000000,2.000000,1.000000,1.000000,3.000000,110.000000,3.000000,500.000000,5.000000,1.000000,1.000000,9.000000e+04,2.000000,80.000000,3.000000,4.600000e+04,4.710000e+04
50%,48.000000,3.000000,1.000000,1.000000,3.000000,160.000000,30.000000,980.000000,7.000000,1.000000,1.000000,1.600000e+05,2.000000,450.000000,5.000000,7.960700e+04,8.010000e+04
75%,48.000000,4.000000,1.000000,1.000000,4.000000,250.000000,80.000000,1500.000000,8.000000,1.000000,1.000000,3.000000e+05,3.000000,840.000000,7.000000,1.264400e+05,1.280000e+05
max,56.000000,20.000000,3.000000,2.000000,12.000000,650.000000,570.000000,8100.000000,22.000000,2.000000,2.000000,5.216000e+06,6.000000,3600.000000,19.000000,2.060000e+06,2.090000e+06


## Part II: Data Wrangling

We now have 16 features and a target column ("Price") with just over 400,000 observations. We will need to clean the data a little further. The next cell will separate the state column into four columns representing each state a method for representing categorical variables known as one-hot encoding.

In [4]:
cols = pd.get_dummies(cols, columns = ["State"])
readable_names = readable_names[1:]
states =["NY", "ND", "TX", "WY"]
readable_names.extend(states)
cols.columns = readable_names

The documentation tells us that some of the values in the year built column correspond to a year and some of them a range of years. We want to change it to the age of the house in years since it is a more reasonable range to work with. For values corresponding to a range I have put the age in the midpoint of the range.

In [5]:
ybl = cols.Year_Built

ybl.loc[ybl == 1.0] = 90
ybl.loc[ybl == 2.0] = 70
ybl.loc[ybl == 3.0] = 60
ybl.loc[ybl == 4.0] = 50
ybl.loc[ybl == 5.0] = 40
ybl.loc[ybl == 6.0] = 30
ybl.loc[ybl == 7.0] = 20
ybl.loc[ybl == 8.0] = 13
ybl.loc[ybl == 9.0] = 10
ybl.loc[ybl == 10.0] = 9
ybl.loc[ybl == 11.0] = 8
ybl.loc[ybl == 12.0] = 7
ybl.loc[ybl == 13.0] = 6
ybl.loc[ybl == 14.0] = 5
ybl.loc[ybl == 15.0] = 4
ybl.loc[ybl == 16.0] = 3
ybl.loc[ybl == 17.0] = 2
ybl.loc[ybl == 18.0] = 1
ybl.loc[ybl == 19.0] = 0
ybl.head()

/Users/hunter/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0     60.0
5     40.0
12    60.0
15    90.0
18     6.0
Name: Year_Built, dtype: float64

In [6]:
cols.rename(columns={"Year_Built":"Age"}, inplace = True)

The columns 'ADJHSG' and 'ADJINC' in the original dataset adjust monetary values in each record for inflation. We will want to adjust the data accordingly so that each record is given in 2015 dollars.

In [7]:
for col in ["Monthly_Electric", "Monthly_Gas", "Yearly_Insurance_Cost", "Yearly_Water"]:
    cols.loc[:,col] *= dat.loc[cols.index, 'ADJHSG'] * 10**-6
    
for col in ["Family_Income", "Household_Income"]:
    cols.loc[:,col] *= dat.loc[cols.index, 'ADJINC'] * 10**-6

## Part III: Exploratory Analysis

Many machine learning algorithms are improved by normalizing the input data. scikit-learn has a handy function to do this. First we will want to separate the features from our target column.

In [8]:
cols.reset_index(drop=True, inplace=True)
Y = cols.Price
X_names = [c for c in cols.columns if c != "Price"]
X = cols[X_names]

In [9]:
X = pd.DataFrame(preprocessing.scale(X))
X.columns = X_names
X.describe()

,Num_People,Lot_Size,Has_Bathtub,Num_Bedrooms,Monthly_Electric,Monthly_Gas,Yearly_Insurance_Cost,Num_Rooms,Has_Hot_Water,Has_Sink,Num_Vehicles,Yearly_Water,Age,Family_Income,Household_Income,NY,ND,TX,WY
count,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05,4.077000e+05
mean,-1.021958e-14,-3.388110e-13,-2.614657e-14,5.171907e-14,1.319594e-14,1.348511e-13,-2.529956e-14,-6.157463e-14,9.060295e-15,-5.833945e-16,-1.084847e-13,3.770523e-14,-5.483500e-14,-1.646544e-15,8.207554e-16,-7.714123e-13,9.903356e-13,1.655999e-13,9.536780e-14
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-8.122866e-01,-5.263009e-01,-4.105418e-02,-3.547572e+00,-1.738066e+00,-6.954202e-01,-1.070900e+00,-2.639081e+00,-4.586939e-02,-4.111408e-02,-2.346045e+00,-9.632052e-01,-1.569245e+00,-1.193754e+00,-1.202319e+00,-7.609020e-01,-1.442985e-01,-1.218105e+00,-1.256330e-01
25%,-8.122866e-01,-5.263009e-01,-4.105418e-02,-3.385562e-01,-7.367983e-01,-6.709525e-01,-6.023771e-01,-8.730647e-01,-4.586939e-02,-4.111408e-02,-2.658352e-01,-8.250802e-01,-8.154670e-01,-5.757474e-01,-5.753915e-01,-7.609020e-01,-1.442985e-01,-1.218105e+00,-1.256330e-01
50%,-7.513454e-02,-5.263009e-01,-4.105418e-02,-3.385562e-01,-2.300062e-01,-3.509025e-01,-1.446626e-01,9.943316e-03,-4.586939e-02,-4.111408e-02,-2.658352e-01,-1.976818e-01,-6.168898e-02,-2.493742e-01,-2.486867e-01,-7.609020e-01,-1.442985e-01,8.209473e-01,-1.256330e-01
75%,6.620175e-01,-5.263009e-01,-4.105418e-02,7.311155e-01,5.492335e-01,2.427408e-01,3.200538e-01,4.514473e-01,-4.586939e-02,-4.111408e-02,7.742697e-01,4.910944e-01,6.920891e-01,2.138902e-01,2.126729e-01,1.314230e+00,-1.442985e-01,8.209473e-01,-1.256330e-01
max,1.245645e+01,2.986249e+00,2.435806e+01,9.288490e+00,4.223610e+00,5.943828e+00,6.440249e+00,6.632503e+00,2.180103e+01,2.432257e+01,3.894584e+00,5.458308e+00,1.822756e+00,1.913584e+01,1.938286e+01,1.314230e+00,6.930077e+00,8.209473e-01,7.959695e+00


In [10]:
print(Y.min())
print(sum(Y < 10000))

100.0
6108


From the last cell, there are quite a few observations where the home is valued at less than 10,000 USD, with the lowest price being only 100 USD. Intuitively that is much lower than we expect the price of a home to be. We will revisit these records soon. Let's split the data into training and testing sets and see how a simple linear regression performs.

In [12]:
#split into test and train
ttsplit = model_selection.train_test_split
X_train, X_test, y_train, y_test = ttsplit(X, Y, test_size = 0.3, random_state = 4)
#the random state here is chosen explicitly so that the code is reproducible

In [13]:
%%time

lm = linear_model.LinearRegression()
lm.fit(X_train, y_train)

CPU times: user 839 ms, sys: 108 ms, total: 947 ms
Wall time: 1.23 s


In [14]:
%%time

lm.score(X_test, y_test)

CPU times: user 32.4 ms, sys: 9.6 ms, total: 42 ms
Wall time: 74.8 ms


0.37621595739879499

The linear classifier is performing poorly, at less than 40% accurate. In order to see where the problem is, I want to see if there is some group of observations with poor predictability.

I first want to see if the extremely low priced (< $10,000) homes have any predictive power or if they are possibly reported erroneously. Since extremely low prices are quite low in proportion to the total dataset I am undersampling the larger class.

In [15]:
ythis = Y[Y < 10000].copy()
ythis = pd.concat([ythis, y_train[y_train >= 10000].head(len(ythis)).copy()])
ythis[ythis < 10000] = 1
ythis[ythis > 1] = 0

xthis = X.loc[ythis.index, :]

#train/test split
xtrainthis, xtestthis, ytrainthis, ytestthis= ttsplit(xthis,
                                                      ythis,
                                                      test_size = .3,
                                                      random_state=4)
clf = RandomForestClassifier(n_estimators = 50, n_jobs = -1)
clf.fit(xtrainthis, ytrainthis)
clf.score(xtestthis, ytestthis)

0.91405184174624832

In [16]:
cols.loc[cols.Price < 10000, states].describe()

,NY,ND,TX,WY
count,6108.000000,6108.000000,6108.000000,6108.000000
mean,0.292731,0.031762,0.650458,0.025049
std,0.455053,0.175379,0.476864,0.156287
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


Apparently there is a relatively large level of predictability with the extremely low priced homes, with a correct classification 89.9% of the time. The extremely low priced observations also do not appear to be concentrated in any one state. Compare the distribution of extremely low priced homes above with the next cell showing distribution of all records.

In [17]:
cols.loc[:,states].describe()

,NY,ND,TX,WY
count,407700.000000,407700.000000,407700.000000,407700.000000
mean,0.366676,0.020397,0.597388,0.015538
std,0.481898,0.141355,0.490425,0.123681
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


Let's look at how well we can predict low (< 100,000 USD), medium (100,000-500,000 USD), and high (> 500,000 USD) priced homes.

In [18]:
sum(Y < 100000)/len(Y)

0.28621535442727497

Approximately 29% of our dataset contains observations with home value under 10,000.

In [20]:
ythis = Y[Y < 100000].copy()
ythis = pd.concat([ythis, y_train[y_train >= 100000].head(len(ythis)).copy()])
ythis[ythis < 100000] = 1
ythis[ythis > 1] = 0

xthis = X.loc[ythis.index, :]

xtrainthis, xtestthis, ytrainthis, ytestthis= ttsplit(xthis,
                                                      ythis,
                                                      test_size = .3,
                                                      random_state=4)
clf = RandomForestClassifier(n_estimators = 50, n_jobs = -1)
clf.fit(xtrainthis, ytrainthis)
clf.score(xtestthis, ytestthis)

0.81326591824492245

The low priced homes are correctly classified approximately 81% of the time.

In [21]:
sum((Y>=100000) & (Y<500000))/len(Y)

0.60302918812852591

The majority of the homes are between \$100,000 and \$500,000. We will not want to undersample here.

In [22]:
ythis = y_train.copy()
ythis[(ythis>=100000) & (ythis<500000)] = 1
ythis[ythis > 1] = 0

xthis = X_train.loc[ythis.index, :]

xtrainthis, xtestthis, ytrainthis, ytestthis= ttsplit(xthis,
                                                      ythis,
                                                      test_size = .3,
                                                      random_state=4)
clf = RandomForestClassifier(n_estimators = 50, n_jobs = -1)
clf.fit(xtrainthis, ytrainthis)
clf.score(xtestthis, ytestthis)

0.75061027599658947

In [23]:
sum(Y >= 500000)/len(Y)

0.11075545744419917

In [24]:
ythis = Y[Y >= 500000].copy()
ythis = pd.concat([ythis, y_train[y_train < 500000].head(len(ythis)).copy()])
ythis[ythis >= 500000] = 1
ythis[ythis > 1] = 0

xthis = X.loc[ythis.index, :]

In [25]:
xtrainthis, xtestthis, ytrainthis, ytestthis= ttsplit(xthis,
                                                      ythis,
                                                      test_size = .3,
                                                      random_state=4)

In [26]:
%%time

clf = RandomForestClassifier(n_estimators = 50, n_jobs = -1)
clf.fit(xtrainthis, ytrainthis)

CPU times: user 8.69 s, sys: 339 ms, total: 9.03 s
Wall time: 5.53 s


In [27]:
%%time
clf.score(xtestthis, ytestthis)

CPU times: user 562 ms, sys: 53.5 ms, total: 616 ms
Wall time: 441 ms


0.83541874284870632

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [29]:
%%time

RFreg = RandomForestRegressor(n_jobs = -1)
params = {'n_estimators':[10,20,50,100], 'max_features':['sqrt','log2']}
reg = model_selection.GridSearchCV(estimator=RFreg, param_grid=params, cv=5)
reg.fit(X_train, y_train)

CPU times: user 44min 25s, sys: 2min 32s, total: 46min 58s
Wall time: 36min 51s


In [30]:
%%time
print("Train score:", reg.score(X_train, y_train))
print("Test score:", reg.score(X_test, y_test))

Train score: 0.926673255799
Test score: 0.454487999914
CPU times: user 44.7 s, sys: 10.6 s, total: 55.4 s
Wall time: 44.4 s


This regressor is incredibly overfit, as the test score is nearly 50% lower than the train score. Further work is necessary to get a less overfitted model. First I want to see if there are any differences in the class balance in the testing and training sets.

In [31]:
print("Training set extreme lows:", sum(y_train < 10000))
print("Expected test set extreme lows:", sum(y_train < 10000) * 3/7)
print("Actual test set extreme lows:", sum(y_test < 10000))
print("Training set lows:", sum(y_train < 100000))
print("Expected test set extreme lows:", sum(y_train < 100000) * 3/7)
print("Actual test set lows:", sum(y_test < 100000))
print("Training set medium:", sum((y_train >= 10000) & (y_train < 500000)))
print("Expected test set extreme lows:", sum((y_train >= 10000) & (y_train < 500000)) * 3/7)
print("Actual test set medium:", sum((y_test >= 10000) & (y_test < 500000)))
print("Training set high:", sum(y_train >= 500000))
print("Expected test set extreme lows:", sum(y_train >= 500000) * 3/7)
print("Actual test set high:", sum(y_test >= 500000))

Training set extreme lows: 4270
Expected test set extreme lows: 1830.0
Actual test set extreme lows: 1838
Training set lows: 81674
Expected test set extreme lows: 35003.1428571
Actual test set lows: 35016
Training set medium: 249280
Expected test set extreme lows: 106834.285714
Actual test set medium: 107157
Training set high: 31840
Expected test set extreme lows: 13645.7142857
Actual test set high: 13315


Apparently there is not a class imbalance in the training and test set. Perhaps this problem is not appropriate for regression. Let's try a multiclass regressor to see how well it scores all of the extremely low, low, medium and high price homes.

In [32]:
Y[Y < 10000] = 1
Y[(Y > 1) & (Y < 100000)] = 2
Y[(Y > 2) & (Y < 500000)] = 3
Y[Y > 3] = 4

/Users/hunter/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/hunter/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/hunter/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/hunter/anaconda/lib/pytho

In [34]:
X_train, X_test, y_train, y_test = ttsplit(X, Y, test_size = 0.3, random_state = 4)

In [35]:
%%time

clf = RandomForestClassifier(n_estimators = 50, n_jobs = -1)
clf.fit(X_train, y_train)

CPU times: user 1min 23s, sys: 1.95 s, total: 1min 25s
Wall time: 53.5 s


In [36]:
%%time

print("Train score:", clf.score(X_train, y_train))
print("Test score:", clf.score(X_test, y_test))

Train score: 0.999772241494
Test score: 0.736260322132
CPU times: user 17.5 s, sys: 2.11 s, total: 19.6 s
Wall time: 14.5 s


In [43]:
print(pd.Series(clf.feature_importances_, X_names).sort_values(ascending = False))

Yearly_Insurance_Cost    0.195457
Family_Income            0.140632
Household_Income         0.120729
Yearly_Water             0.092986
Monthly_Electric         0.092630
Monthly_Gas              0.081120
Age                      0.068537
Num_Rooms                0.056200
Num_People               0.033601
Num_Vehicles             0.032230
Num_Bedrooms             0.031068
Lot_Size                 0.016792
NY                       0.016378
TX                       0.016297
WY                       0.002128
ND                       0.002115
Has_Hot_Water            0.000452
Has_Sink                 0.000342
Has_Bathtub              0.000306
dtype: float64
